In [1]:
model_name="infgrad/jasper_en_vision_language_v1"

In [2]:
import os

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

os.environ['WANDB_DISABLED'] = "true"

In [3]:
# !gdown 1qq5gDuEAqWnoviRBDYfbprLFD-MPQyNo
# !gdown 1mq9CWT9MXsH1Y-ihxVL4uEzufQjpBoer
!gdown 1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql

Downloading...
From: https://drive.google.com/uc?id=1EI12smTmdi4Mlu_lLNULN6p6g9dYEWql
To: /content/processed_data.zip
100% 6.00M/6.00M [00:00<00:00, 32.1MB/s]


In [4]:
!rm -rf processed_data

!unzip -q processed_data.zip

In [5]:
!pip install -q transformers[torch] accelerate -U

!pip install -q datasets

!pip install -q transformers

!pip install -q iterative-stratification

!pip install -q auto-gptq optimum bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 33.9 MB/s eta 0:00:00
   ━━━━━

In [6]:
import numpy as np

import pandas as pd

import os

from os import path

import glob

from tqdm.notebook import tqdm

from tqdm import tqdm

import string

import copy

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer,IntervalStrategy,BitsAndBytesConfig

from datasets import Dataset,load_dataset, DatasetDict

# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay



from sklearn.model_selection import train_test_split,KFold

from sklearn.metrics import classification_report,accuracy_score, f1_score

from sklearn.metrics import precision_recall_fscore_support

from sklearn.utils import shuffle

from sklearn.preprocessing import LabelEncoder

from scipy.special import expit

from peft import prepare_model_for_kbit_training

from peft import LoraConfig, get_peft_model,PeftModel


In [7]:
import torch

import torch.nn as nn

import torch.nn.functional as F

In [8]:


models_list_3=[
        model_name,
          ]



models_name_3=[
        model_name,

]


In [9]:
kind="train"

task="track_a"

# langs=["afr","amh","deu","eng","oro","ptbr","rus","som","sum","tir"]
# labels=['Anger','Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
# langs=["amh","arq","ary","chn","deu","esp","hau","hin","ibo","kin","mar","orm","pcm","ptbr","ptmz","ron","rus","som","sun","swa","swe","tat","tir","ukr","vmw","yor","ind","jav","xho","zul"]
langs=["eng"]
# langs=["afr"]


# processed_path=f"processed_data/{kinds[0]}/{tasks[0]}/{langs[3]}.csv"



# train_data=pd.read_csv(processed_path)


In [10]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)




In [11]:
train_data=all_csv


In [12]:
train_data=train_data.fillna(0)

In [13]:
float_columns = train_data.select_dtypes(include=['float64']).columns
# Convert those columns to int
train_data[float_columns] = train_data[float_columns].astype('int')

In [14]:
train_data = train_data[

train_data['clean_message'].isnull()==False]

train_data.reset_index()

,index,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,0,eng_train_track_a_00001,0,1,0,0,1,eng,colorado middle of nowhere
1,1,eng_train_track_a_00002,0,1,0,0,0,eng,this involved swimming a pretty large lake tha...
2,2,eng_train_track_a_00003,0,1,0,1,0,eng,it was one of my most shameful experiences
3,3,eng_train_track_a_00004,0,0,0,0,0,eng,after all i had vegetables coming out my ears ...
4,4,eng_train_track_a_00005,0,1,0,1,1,eng,then the screaming started
...,...,...,...,...,...,...,...,...,...
2763,2763,eng_train_track_a_02764,0,0,1,0,1,eng,she cants her hip against my waist into my sid...
2764,2764,eng_train_track_a_02765,0,0,0,0,0,eng,i then did the dishes whitened my teeth watche...
2765,2765,eng_train_track_a_02766,0,0,0,0,1,eng,it just kind of gradually vanished over a coup...
2766,2766,eng_train_track_a_02767,0,1,0,0,0,eng,i didnt look out of my hands


In [15]:
label_columns =['anger', 'fear', 'joy', 'sadness', 'surprise']
# label_columns =['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']


In [16]:
train_data['lang'].value_counts()

,count
lang,
eng,2768


In [17]:
sum(train_data['lang'].isna())

0

In [18]:
nan_rows = train_data[train_data.isna().any(axis=1)]
nan_rows

,id,anger,fear,joy,sadness,surprise,lang,clean_message


In [19]:
train_data['clean_message']=train_data['clean_message'].astype(str)

In [20]:
maxi=train_data['clean_message'].apply(len).max()
maxi

441

In [21]:
train_data[train_data['clean_message'].apply(len) == maxi]

,id,anger,fear,joy,sadness,surprise,lang,clean_message
2391,eng_train_track_a_02392,0,0,1,0,0,eng,its most famous picture is picassos guernica i...


In [22]:
train_data[train_data['clean_message'].apply(len) == maxi]["clean_message"].values

array(['its most famous picture is picassos guernica inspired by the civil war spain was enduringit has a selection of amazing surrealist works by dali and a number by joan miro some of which were laughably simplistic to my eyes and a selection of art from the 1940s to present in its permanent collectionfor its temporary exhibitions it had one on digital art and new mediamy favourite works were the series of ` mirrors by daniel rozin from israel'],
      dtype=object)

In [23]:
# shuffle data
train_data=train_data.sample(frac=1)
# train_data=train_data.sample(100,random_state=42)


In [24]:
# train valid split
# train,valid=train_test_split(train_data,test_size=0.2,random_state=42)

In [25]:
x_train, y_train = train_data['clean_message'].values.tolist(), train_data[label_columns].values.tolist()

In [26]:
y_train_df = pd.DataFrame(y_train, columns=label_columns)

In [27]:
#update label column name

label_distribution = y_train_df.apply(pd.Series.value_counts).T.fillna(0).astype(int)

label_distribution.columns = ['count_0', 'count_1']
label_distribution['sum'] = label_distribution['count_0'] + label_distribution['count_1']
print(label_distribution)

          count_0  count_1   sum
anger        2435      333  2768
fear         1157     1611  2768
joy          2094      674  2768
sadness      1890      878  2768
surprise     1929      839  2768


In [28]:
device="cuda" if torch.cuda.is_available() else 'cpu'

In [29]:
from sklearn.utils.class_weight import compute_class_weight

counts_0 = label_distribution['count_0'].to_numpy()
counts_1 = label_distribution['count_1'].to_numpy()

# Compute class-specific weights for each label (each class)
class_weights = []
for i in range(len(counts_0)):
    weight = compute_class_weight('balanced', classes=np.array([0, 1]), y=[0] * counts_0[i] + [1] * counts_1[i])
    class_weights.append(weight)

# Convert the list of weights to a tensor
class_weights_tensor = torch.tensor([w[1] for w in class_weights], dtype=torch.float,device=device)
class_weights_tensor

tensor([4.1562, 0.8591, 2.0534, 1.5763, 1.6496], device='cuda:0')

In [30]:
MAX_LENGTH = 512

metrics_df = pd.DataFrame(columns=["Models_name", "Precision", "Recall", "F1_score", "Accuracy"])

confusion_matrices = []

In [31]:
quantization_config = BitsAndBytesConfig(

    load_in_4bit=True,

    bnb_4bit_use_double_quant=True,

    bnb_4bit_quant_type="nf4",

    bnb_4bit_compute_dtype=torch.bfloat16,

)

In [32]:
models_list=models_list_3

models_name=models_name_3

x=models_list[0]

In [33]:
token=""

import huggingface_hub

if "gemma" in x or "Phi" in x:

  huggingface_hub.login(token=token)

In [34]:
USE_LORA=False

In [35]:
!pip install -q sentence_transformers

In [36]:
from transformers import AutoModel
from sentence_transformers import SentenceTransformer

In [38]:


torch.cuda.empty_cache()



print(x)
use_gpu = torch.cuda.is_available()
embedding_model = SentenceTransformer(
        x,
        trust_remote_code=True,
        device="cpu" if not use_gpu else "cuda",
        model_kwargs={
            "torch_dtype": torch.bfloat16 if use_gpu else torch.float32,
            # "attn_implementation": "sdpa"
        },
        # vector_dim must be 12288, 1024, 512, 256
        ## 1024 is recommended
        # set is_text_encoder 'True', if you do not encode image
        # config_kwargs={"is_text_encoder": True, "vector_dim": 1024},
    )
    # We can reduce the max_seq_length from the default of 2048 for faster encoding
embedding_model.max_seq_length = MAX_LENGTH

infgrad/jasper_en_vision_language_v1


model.safetensors:  99%|#########9| 3.95G/3.99G [00:00<?, ?B/s]

Some weights of the model checkpoint at infgrad/jasper_en_vision_language_v1 were not used when initializing JasperVL: {'vision_model.vision_model.encoder.layers.19.mlp.fc1.weight', 'vision_model.vision_model.encoder.layers.1.mlp.fc2.bias', 'vision_model.vision_model.head.layernorm.weight', 'vision_model.vision_model.encoder.layers.9.layer_norm2.weight', 'vision_model.vision_model.encoder.layers.24.layer_norm1.weight', 'vision_model.vision_model.encoder.layers.20.mlp.fc2.weight', 'vision_model.vision_model.encoder.layers.12.self_attn.v_proj.bias', 'vision_model.vision_model.head.probe', 'vision_model.vision_model.post_layernorm.bias', 'vision_model.vision_model.encoder.layers.15.self_attn.k_proj.bias', 'vision_model.vision_model.encoder.layers.16.mlp.fc1.bias', 'vision_model.vision_model.encoder.layers.6.layer_norm2.bias', 'vision_model.vision_model.encoder.layers.15.self_attn.out_proj.bias', 'vision_model.vision_model.encoder.layers.15.mlp.fc1.weight', 'vision_model.vision_model.encod

tokenizer_config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

In [39]:
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()

    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear4bit'
        if "Linear4bit" in str(type(module)):
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]

            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)
find_target_modules(embedding_model)

[]

In [40]:
# a=

In [41]:
embedding_model.eval() # embedding_model in training mode (dropout modules are activated)

if USE_LORA:

  # enable gradient check pointing

  embedding_model.gradient_checkpointing_enable()

  target_modules_dict={

      "microsoft/Phi-3.5-mini-instruct":'all-linear',
      "unsloth/Llama-3.2-1B-Instruct":["q_proj","k_proj","v_proj"],
      "unsloth/Qwen2.5-3B-Instruct-bnb-4bit":["q_proj","k_proj","v_proj"],
  }

  target_modules= target_modules_dict.get(x,["q_proj"])



  # enable quantized training

  embedding_model = prepare_model_for_kbit_training(embedding_model)

  config = LoraConfig(

    r=8,

    lora_alpha=32,

    # target_modules=["k_proj"],

    target_modules = target_modules,#TODOOOOOOOOOOOOO

    lora_dropout=0.1,

    bias="none",

    task_type="SEQ_CLS",

  )

  # LoRA trainable version of model

  embedding_model = get_peft_model(embedding_model, config)



  # trainable parameter count

  embedding_model.print_trainable_parameters()

device="cuda" if torch.cuda.is_available() else 'cpu'

In [42]:
device

'cuda'

In [43]:
embedding_model = embedding_model.to(device)

for param in embedding_model.parameters():
    param.data = param.data.contiguous()


In [50]:

# Custom Dataset for batching
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize a single example
        return self.texts[idx]


In [51]:
from torch.utils.data import DataLoader

In [52]:

# Batch extraction function
def extract_embeddings(texts, model, batch_size=32):
    dataset = TextDataset(texts)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    embeddings = []
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting embeddings"):
            # Move batch to device
            # input_ids = torch.Tensor(batch['input_ids']).squeeze().to(device)
            # attention_mask = torch.Tensor(batch['attention_mask']).squeeze().to(device)

            # Forward pass through the model
            cls_embeddings = model.encode(batch)
            embeddings.append(cls_embeddings)

    return np.vstack(embeddings)

In [53]:
# Extract embeddings for train and validation datasets
x_train_embeddings = extract_embeddings(x_train, embedding_model, batch_size=32)

Extracting embeddings: 100%|██████████| 87/87 [02:40<00:00,  1.85s/it]


In [54]:
np.save("x_train_embeddings.npy",x_train_embeddings)

In [55]:
# Flatten labels for XGBoost (multi-label classification)
y_train_flat = np.array(y_train)

In [56]:
import xgboost as xgb

In [57]:
# xgb_models = {}

# for i, label in enumerate(label_columns):
#     print(f"Training XGBoost for label: {label}")
#     # Use the computed class weights for the positive class
#     scale_pos_weight = class_weights_tensor[i].item()
#     # Initialize and train XGBoost classifier
#     xgb_model = xgb.XGBClassifier(
#         objective='binary:logistic',
#         eval_metric='logloss',
#         use_label_encoder=False,
#         n_estimators=100,
#         learning_rate=0.1,
#         max_depth=6,
#         tree_method='hist',
#         scale_pos_weight=scale_pos_weight,  # Apply the class weight here
#         device=device
#     )
#     xgb_model.fit(x_train_embeddings, y_train_flat[:, i])
#     xgb_models[label] = xgb_model

In [58]:
# Save the XGBoost models if needed
# for label, model in xgb_models.items():
#     model.save_model(f"xgb_model_{label}.json")

In [59]:
# import pickle
# # Save the XGBoost models if needed
# for label, model in xgb_models.items():
#     with open(f"xgb_model_{label}.pkl", 'wb') as f:
#       pickle.dump(model, f)

In [60]:
# !zip xgb_models_pkl.zip xgb_model_*.pkl

In [61]:
from sklearn.svm import SVC

In [62]:
svm_models = {}

for i, label in enumerate(label_columns):
    print(f"Training SVM for label: {label}")

    # Use the computed class weights for balancing the classes
    class_weight = {0: 1, 1: class_weights_tensor[i].item()}  # Adjust the class weights

    svm_model = SVC(class_weight={0: 1, 1: class_weights_tensor[i].item()})

    svm_model.fit(x_train_embeddings, y_train_flat[:, i])
    svm_models[label] = svm_model

Training SVM for label: anger
Training SVM for label: fear
Training SVM for label: joy
Training SVM for label: sadness
Training SVM for label: surprise


In [63]:
import pickle
# Save the svm models if needed
for label, model in svm_models.items():
    with open(f"svm_model_{label}.pkl", 'wb') as f:
      pickle.dump(model, f)

In [64]:
!zip svm_models_pkl.zip svm_model_*.pkl

  adding: svm_model_anger.pkl (deflated 75%)
  adding: svm_model_fear.pkl (deflated 75%)
  adding: svm_model_joy.pkl (deflated 75%)
  adding: svm_model_sadness.pkl (deflated 75%)
  adding: svm_model_surprise.pkl (deflated 75%)


In [65]:
kind="dev"

In [66]:
all_csv=pd.DataFrame()

for lang in langs:

    processed_path=f"processed_data/{kind}/{task}/{lang}.csv"
    # if kind=="dev":
    #    processed_path=processed_path.replace(lang,lang+task[-2:])
    if not os.path.isfile(processed_path):

      print("not found:",processed_path)

      continue
    train_data=pd.read_csv(processed_path)

    train_data.columns = train_data.columns.str.lower()

    all_csv = pd.concat([all_csv, train_data],ignore_index=True)
dev_data=all_csv
dev_data=dev_data.fillna(0)
float_columns = dev_data.select_dtypes(include=['float64']).columns

# Convert those columns to int
dev_data[float_columns] = dev_data[float_columns].astype('int')
dev_data = dev_data[dev_data['clean_message'].isnull()==False]
dev_data.reset_index()


,index,id,anger,fear,joy,sadness,surprise,lang,clean_message
0,0,eng_dev_track_a_00001,0,0,0,0,0,eng,older sister 23 at the time is a scumbag stacy
1,1,eng_dev_track_a_00002,0,0,0,0,0,eng,and i laughed like this garhahagar because my ...
2,2,eng_dev_track_a_00003,0,0,0,0,0,eng,it overflowed and brown shitty diarrhea water ...
3,3,eng_dev_track_a_00004,0,0,0,0,0,eng,its very dark and foggy
4,4,eng_dev_track_a_00005,0,0,0,0,0,eng,then she tried to like have sex withstrangle e...
...,...,...,...,...,...,...,...,...,...
111,111,eng_dev_track_a_00112,0,0,0,0,0,eng,my heart was beating fast from excitement
112,112,eng_dev_track_a_00113,0,0,0,0,0,eng,a fraying rope stretches down from the rafters
113,113,eng_dev_track_a_00114,0,0,0,0,0,eng,so i cried my eyes out and did the drawing
114,114,eng_dev_track_a_00115,0,0,0,0,0,eng,never been so close to a group asswooping in m...


In [67]:
x_dev, y_dev = dev_data['clean_message'].values.tolist(), dev_data[label_columns].values.tolist()

In [68]:
y_dev_df = pd.DataFrame(y_dev, columns=label_columns)

In [69]:
x_dev_embeddings = extract_embeddings(x_dev, embedding_model, batch_size=32)

Extracting embeddings: 100%|██████████| 4/4 [00:05<00:00,  1.44s/it]


In [70]:
y_dev_flat = np.array(y_dev)

In [71]:
# Predict labels for the test data
test_predictions = {}
for label in label_columns:
    print(f"Predicting for label: {label}")
    test_predictions[label] = svm_models[label].predict(x_dev_embeddings)

Predicting for label: anger
Predicting for label: fear
Predicting for label: joy
Predicting for label: sadness
Predicting for label: surprise


In [72]:
predictions_df = pd.DataFrame(test_predictions)
dev_data.reset_index(inplace=True)

In [73]:
output_df = pd.concat([dev_data[['id','lang']], predictions_df], axis=1)

In [74]:
# Create directory if it doesn't exist
output_dir = task
os.makedirs(output_dir, exist_ok=True)

# Iterate through the unique languages in the 'lang' column
for lang in output_df['lang'].unique():
    # Filter rows for the current language
    lang_df = output_df[output_df['lang'] == lang]
    lang_df.drop(columns=['lang'],inplace=True)

    # Define the output file path for the current language
    lang_csv_file = os.path.join(output_dir, f"pred_{lang}.csv")

    # Sort the DataFrame by the 'id' column
    lang_df.sort_values(by='id', inplace=True)

    # Save the filtered DataFrame to a CSV file
    lang_df.to_csv(lang_csv_file, index=False)

    # Print a confirmation message
    print(f"Predictions for language '{lang}' saved to {lang_csv_file}")

Predictions for language 'eng' saved to track_a/pred_eng.csv


In [75]:
!zip -r {task}.zip {task}

  adding: track_a/ (stored 0%)
  adding: track_a/pred_eng.csv (deflated 88%)
